In [52]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm


In [53]:
dataset = pd.read_csv("youtube_parsed_dataset.csv")

In [69]:
pip install torchtext

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Pranay\\anaconda3\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.




     ---------------------------------------- 1.9/1.9 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 14.0 MB/s eta 0:00:00
     ------------------------------------- 198.6/198.6 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1


ModuleNotFoundError: No module named 'torchtext'

In [54]:
dataset[dataset['oh_label'] == 1]

,index,UserIndex,Text,Number of Comments,Number of Subscribers,Membership Duration,Number of Uploads,Profanity in UserID,Age,oh_label
2,2,X3,3:26 hahah my boyfriend showed this song to me...,7,0,3,5,0,43,1
3,3,X2218,dick beyonce fuck y a ass hole you are truely ...,34,0,3,5,0,44,1
21,21,X811,@RossJohn69 STFU with ur god bullshit u dumbas...,36,2,2,5,0,22,1
23,23,X24,Ugh Martin joins DaveChaos in my hatred for bi...,6,1,5,5,0,21,1
25,25,X26,If you ever hit or shoot a cop just say you di...,35,1,7,5,0,62,1
...,...,...,...,...,...,...,...,...,...,...
3420,3425,X3426,Epic Is that lils wanyes bro because they are ...,16,1,2,4,0,14,1
3422,3427,X3428,i dont give a shit who you are. but if it woul...,35,3,2,5,0,14,1
3445,3450,X3451,is that ron jeremy spotting him? Dee Snider yo...,2,0,7,5,0,23,1
3457,3462,X3463,Justin looked like he was enjoying some of Nic...,4,4,3,5,0,111,1


In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

train_df = pd.DataFrame({'Text': dataset['Text'], 'oh_label': dataset['oh_label']})

class_0_samples = train_df[train_df['oh_label'] == 0]
class_1_samples = train_df[train_df['oh_label'] == 1]

num_samples_to_keep = min(len(class_0_samples), len(class_1_samples))

class_0_samples_to_keep = class_0_samples.sample(n=num_samples_to_keep, random_state=42)
class_1_samples_to_keep = class_1_samples.sample(n=num_samples_to_keep, random_state=42)

train_df_balanced = pd.concat([class_0_samples_to_keep, class_1_samples_to_keep])

train_df_balanced = train_df_balanced.sample(frac=1, random_state=42)

train_texts, val_texts, train_labels, val_labels = train_test_split(train_df_balanced['Text'], train_df_balanced['oh_label'], test_size=0.2, random_state=42)

def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

train_texts = train_texts.apply(preprocess_text)
val_texts = val_texts.apply(preprocess_text)

print(train_texts[0])


Nebodyelse Hear Crazy as Screamin HOE everytime say Stupid Hoe Lol nebody see Chris Browns face 204 wen Nicki touchin worked wouldnt lookin Body look tired ah Tatto as Wtf oO Instagram Deze Nutts Yo Mouth Lmao VideoSong made ah Lil Umm mood together motherfucka Wow amazing voice Ive never said anyone


In [58]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [59]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Define training parameters
batch_size = 20
max_length = 128
num_epochs = 10
learning_rate = 2e-5

# Prepare datasets and dataloaders
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [60]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
#         print(batch)
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
      
        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    model.eval()
    val_predictions = []
    val_targets = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation"):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            val_predictions.extend(torch.argmax(logits, dim=1).tolist())
            val_targets.extend(labels.tolist())
            print(val_predictions)

    val_accuracy = accuracy_score(val_targets, val_predictions)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss / len(train_loader)}, Validation Accuracy: {val_accuracy}")

Validation:  11%|████████                                                                | 1/9 [00:09<01:19,  9.97s/it]

[0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1]


Validation:  22%|████████████████                                                        | 2/9 [00:19<01:06,  9.49s/it]

[0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]


Validation:  33%|████████████████████████                                                | 3/9 [00:27<00:55,  9.17s/it]

...

[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]


Validation:  89%|████████████████████████████████████████████████████████████████        | 8/9 [00:43<00:05,  5.51s/it]

[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]


Validation: 100%|████████████████████████████████████████████████████████████████████████| 9/9 [00:45<00:00,  5.09s/it]

[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1]
Epoch 10: Train Loss: 0.041047483244363, Validation Accuracy: 0.6646706586826348


In [26]:
# Example input text
input_text = "@RossJohn69 STFU with ur god bullshit u dumbass @lostinthecrowd5827 everything apparently to the gay dudes -_- Arrraghhhh!!!! I JUST WANT THIS SHIT TO BE RELEASED ALLREADY!!! :( i imagine its hard to rap and sing with cotton mouth my mouth and throaght ge"
input_text = input_text
# Tokenize input text
inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,
    max_length=max_length,
    return_token_type_ids=False,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

print("Predicted class:", predicted_class)


Predicted class: 1


In [27]:
logits

tensor([[-2.5728,  3.1170]])

In [28]:
# Prepare datasets and dataloaders
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)

# Print tokenized text
for i in range(len(train_dataset)):
    sample = train_dataset[i]
    input_ids = sample['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print(f"Tokenized text for sample {i}: {tokens}")

val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)


Tokenized text for sample 0: ['<s>', 'ah', 'h', 'Ġpipe', 'Ġit', 'Ġdown', 'Ġya', 'Ġvegan', 'Ġhumans', 'Ġeat', 'Ġmeat', 'Ġand', 'Ġthats', 'Ġhow', 'Ġhamb', 'urg', 'ers', 'Ġare', 'Ġmade', 'ĠGo', 'Ġlive', 'Ġon', 'Ġa', 'Ġcattle', 'Ġranch', 'Ġfor', 'Ġa', 'Ġyear', 'Ġor', 'Ġtwo', 'Ġand', 'Ġyou', 'll', 'Ġstop', 'Ġthinking', 'Ġthat', 'Ġthats', 'Ġcruel', 'Ġright', 'Ġtrust', 'Ġme', 'Ġhe', 'Ġmade', 'Ġit', 'ĠB', 'ode', 'Ġmay', 'Ġnot', 'Ġbe', 'Ġthe', 'Ġnic', 'est', 'Ġperson', 'Ġon', 'Ġthe', 'Ġplanet', 'Ġbut', 'Ġhe', 'Ġsure', 'Ġas', 'Ġhell', 'Ġcan', 'Ġski', 'Ġso', 'Ġi', 'Ġlike', 'Ġthis', 'Ġsong', 'Ġbut', 'Ġwhat', 'Ġis', 'Ġit', 'Ġabout', 'Ġepic', 'Ġwin', 'ĠY', 'O', 'Ġmade', 'Ġa', 'Ġremix', 'Ġto', 'Ġthis', 'Ġnot', 'Ġtrying', 'Ġto', 'Ġget', 'Ġfans', 'Ġor', 'Ġanything', 'Ġbut', 'ĠIts', 'Ġgood', 'ĠSearch', 'ĠFacebook', 'Ġfor', 'ĠDJ', 'ĠSn', 'are', 'ĠRUN', 'h', 'ope', 'Ġyou', 'Ġlike', 'Ġit', 'Ġcheck', 'Ġout', 'Ġsome', 'Ġof', 'Ġmy', 'Ġother', 'Ġstuff', 'Ġwhile', 'Ġyour', 'Ġat', 'Ġit', 'ĠDJ', 'ĠSn', 'are', 'ĠR

In [44]:
dataset2 = pd.read_csv("twitter_parsed_dataset.csv")
dataset2['Text']

import pandas as pd
from sklearn.model_selection import train_test_split
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

train_df = pd.DataFrame({'Text': dataset2['Text'], 'oh_label': dataset2['oh_label']})

class_0_samples = train_df[train_df['oh_label'] == 0]
class_1_samples = train_df[train_df['oh_label'] == 1]

num_samples_to_keep = min(len(class_0_samples), len(class_1_samples))

class_0_samples_to_keep = class_0_samples.sample(n=num_samples_to_keep, random_state=42)
class_1_samples_to_keep = class_1_samples.sample(n=num_samples_to_keep, random_state=42)

train_df_balanced = pd.concat([class_0_samples_to_keep, class_1_samples_to_keep])

train_df_balanced = train_df_balanced.sample(frac=1, random_state=42)

train_texts, val_texts, train_labels, val_labels = train_test_split(train_df_balanced['Text'], train_df_balanced['oh_label'], test_size=0.2, random_state=42)
print(train_texts[0])
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


@halalflaws @biebervalue @greenlinerzjm I read them in context.No change in meaning. The history of Islamic slavery. https://t.co/xWJzpSodGj


In [37]:
import nltk 

In [38]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Pranay\AppData\Roaming\nltk_data...


True

In [46]:
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

train_texts = train_texts.apply(preprocess_text)
val_texts = val_texts.apply(preprocess_text)

print(train_texts[0])

read contextNo change meaning history Islamic slavery


In [49]:
val_labels.value_counts()


1.0    1072
0.0    1067
Name: oh_label, dtype: int64

In [50]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Define training parameters
batch_size = 128
max_length = 128
num_epochs = 3
learning_rate = 2e-5

# Prepare datasets and dataloaders
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [51]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
#         print(batch)
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
      
        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    model.eval()
    val_predictions = []
    val_targets = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation"):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            val_predictions.extend(torch.argmax(logits, dim=1).tolist())
            val_targets.extend(labels.tolist())
            print(val_predictions)

    val_accuracy = accuracy_score(val_targets, val_predictions)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss / len(train_loader)}, Validation Accuracy: {val_accuracy}")

Validation:   6%|████▏                                                                  | 1/17 [00:36<09:44, 36.53s/it]

[0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1]


Validation:  12%|████████▎                                                              | 2/17 [01:12<09:05, 36.39s/it]

[0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1]


Validation:  18%|████████████▌                                                          | 3/17 [01:48<08:27, 36.28s/it]

...

[0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 

Validation: 100%|██████████████████████████████████████████████████████████████████████| 17/17 [10:00<00:00, 35.31s/it]


[0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 

Epoch 3:   0%|                                                                                  | 0/67 [01:46<?, ?it/s]


KeyboardInterrupt: 